# hardnet_with_dappm

Стандартная сеть [FCHarDNet](https://github.com/PingoLH/FCHarDNet/blob/fc4b854b5cfa01a449bcfaece6bb3c32d84d9e2b/ptsemseg/models/hardnet.py#L254-L359), с блоком [DAPPM](https://github.com/chenjun2hao/DDRNet.pytorch/blob/1daf52a3eeb638f86fd07d2fba3a46c68bca8f15/lib/models/ddrnet_23_slim.py#L94-L175) из [DDRNet](https://arxiv.org/pdf/2101.06085v1.pdf)

In [ ]:
import sys
sys.path.insert(0, '.')
import os
import logging
import math
import random
import time
# import shutil
# from tabulate import tabulate
from datetime import datetime
 
!pip install tensorboardX
from tensorboardX import SummaryWriter
 
import matplotlib.pyplot as plt
 
from tqdm import tqdm
import numpy as np
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
 
from torchsummary import summary
from torchvision.datasets import Cityscapes
from torchvision import transforms

     |████████████████████████████████| 122kB 12.7MB/s 


## Blocks

In [ ]:
class ConvLayer(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel=3, stride=1, dropout=0.1, padding=None, dilation=1):
        super().__init__()
        if padding is None:
            padding = kernel//2 + dilation - 1
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=kernel,
                                          stride=stride, padding=padding, dilation=dilation, bias = False))
        self.add_module('norm', nn.BatchNorm2d(out_channels))
        self.add_module('relu', nn.ReLU(inplace=True))

    def forward(self, x):
        return super().forward(x)

class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

    def forward(self, x, skip, concat=True):
        out = F.interpolate(
                x,
                size=(skip.size(2), skip.size(3)),
                mode="bilinear",
                align_corners=True,
                            )
        if concat:                            
          out = torch.cat([out, skip], 1)
          
        return out

In [ ]:
class my_HarDBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, grmul, n_layers, keepBase=False):
        super().__init__()
        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.grmul = grmul
        self.links = []
        self.out_channels = 0
        self.keepBase = keepBase
        self.layers = nn.ModuleList([])
        for i in range(n_layers):
            out_ch, in_ch, el_links = self.get_links(i+1)
            self.links.append(el_links)
            self.layers.append(ConvLayer(in_ch, out_ch))
            if (i % 2 == 0) or (i == n_layers - 1):
                self.out_channels += out_ch

    def __out_ch(self, layer_id): 
        out_ch = self.growth_rate
        for i in range(1, int(math.log2(layer_id))+1):
            if layer_id % 2**i == 0:
                out_ch = out_ch*self.grmul
        return int(int(out_ch + 1) / 2) * 2

    def get_links(self, layer_id):
        in_ch = 0
        links_ = []
        for i in range(int(math.log2(layer_id))):
            diff = 2**i
            if (layer_id % diff == 0) and layer_id - diff > 0:
                in_ch += self.__out_ch(layer_id - diff)
                links_.append(layer_id - diff)
        if math.log2(layer_id).is_integer():
            in_ch += self.in_channels
            links_.append(0)
        return self.__out_ch(layer_id), in_ch, links_

    def forward(self, x):
        data = [x]
        for layer in range(len(self.layers)):
            layer_input = []
            for link in self.links[layer]:
                layer_input.append(data[link])
            in_ = layer_input[0] if len(layer_input) == 1 \
                else torch.cat(layer_input, dim=1)
            data.append(self.layers[layer](in_))
        t = len(data)
        out = []
        for i in range(t):
            if (i % 2 == 1) or (self.keepBase and i == 0) or (i == t-1):
                out.append(data[i])
        return torch.cat(out, dim=1)
    
    def get_out_ch(self):
        return self.out_channels

In [ ]:
bn_mom = 0.1

class DAPPM(nn.Module):
    def __init__(self, inplanes, branch_planes, outplanes):
        super(DAPPM, self).__init__()
        self.scale1 = nn.Sequential(nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale2 = nn.Sequential(nn.AvgPool2d(kernel_size=9, stride=4, padding=4),
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale3 = nn.Sequential(nn.AvgPool2d(kernel_size=17, stride=8, padding=8),
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale4 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale0 = nn.Sequential(
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.process1 = nn.Sequential(
                                    nn.BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process2 = nn.Sequential(
                                    nn.BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process3 = nn.Sequential(
                                    nn.BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process4 = nn.Sequential(
                                    nn.BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )        
        self.compression = nn.Sequential(
                                    nn.BatchNorm2d(branch_planes * 5, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes * 5, outplanes, kernel_size=1, bias=False),
                                    )
        self.shortcut = nn.Sequential(
                                    nn.BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=False),
                                    )

    def forward(self, x):

        #x = self.downsample(x)
        width = x.shape[-1]
        height = x.shape[-2]
        x_list = []

        x_list.append(self.scale0(x))
        x_list.append(self.process1((F.interpolate(self.scale1(x),
                        size=[height, width],
                        mode='bilinear')+x_list[0])))
        x_list.append((self.process2((F.interpolate(self.scale2(x),
                        size=[height, width],
                        mode='bilinear')+x_list[1]))))
        x_list.append(self.process3((F.interpolate(self.scale3(x),
                        size=[height, width],
                        mode='bilinear')+x_list[2])))
        x_list.append(self.process4((F.interpolate(self.scale4(x),
                        size=[height, width],
                        mode='bilinear')+x_list[3])))
       
        out = self.compression(torch.cat(x_list, 1)) + self.shortcut(x)
        return out 


## Model

In [ ]:
# dapp in low and right U part and replace out to result of dapp

class hardnet_m(nn.Module):
    def __init__(self, n_classes=19, use_dappm=False):
        super(hardnet_m, self).__init__()
        self.n_classes = n_classes
        self.use_dappm = use_dappm

        spp_channels = 128
        first_ch  = [16,24,32,48]
        ch_list = [  64, 96, 160, 224, 320]
        grmul = 1.7
        gr       = [  10,16,18,24,32]
        n_layers = [   4, 4, 8, 8, 8]

        base = [
            ConvLayer(          3, first_ch[0], stride=2, dilation=1),
            ConvLayer(first_ch[0], first_ch[1], stride=1, dilation=2),
            ConvLayer(first_ch[1], first_ch[2], stride=2, dilation=1),
            ConvLayer(first_ch[2], first_ch[3], stride=1, dilation=2)
        ]

        self.base = nn.ModuleList(base)

        blks_count = len(n_layers)
        ch = first_ch[-1]
        skip_connection_channel_counts = []
        self.shortcut_layers = []
        for i in range(blks_count):
            block = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            ch = block.get_out_ch()
            skip_connection_channel_counts.append(ch)
            
            self.base.append(block)
            if i < blks_count-1:
                self.shortcut_layers.append(len(self.base)-1)
            
            self.base.append(ConvLayer(ch, ch_list[i], kernel=1))
            ch = ch_list[i]
            if (i < blks_count - 1):
                self.base.append(nn.AvgPool2d(kernel_size=2, stride=2))

        self.conv1x1_up = nn.ModuleList()
        self.transUp = nn.ModuleList()
        self.denseUp = nn.ModuleList()

        blks_count -= 1
        self.blks_count = blks_count 

        for i in range(blks_count - 1, -1, -1):
            self.transUp.append(TransitionUp(ch, ch))
            ch = ch + skip_connection_channel_counts[i]
            self.conv1x1_up.append(ConvLayer(ch, ch//2, kernel=1))
            ch = ch//2
            blk = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            self.denseUp.append(blk)
            ch = blk.get_out_ch()
        
        if self.use_dappm:
            # end of network, right path of U
            channels_low  = self.denseUp[-1].get_out_ch()
            # mid of network, low part of U
            channels_high = self.base[self.shortcut_layers[-1]].get_out_ch() 

            self.spp_low  = DAPPM(channels_low,  spp_channels, channels_low )
            self.spp_high = DAPPM(channels_high, spp_channels, channels_high)

        self.finalConv = nn.Conv2d(ch, n_classes, kernel_size=1, 
                                   stride=1, padding=0, bias=True)

    def forward(self, x):
        size_in = x.size()
        out = x
        save_shortcut = []
        for i in range(len(self.base)):
            out = self.base[i](out)
            if i in self.shortcut_layers:
                if i == self.shortcut_layers[-1] and self.use_dappm:
                    out = self.spp_high(out)
                save_shortcut.append(out)

        
        for i in range(self.blks_count):
            skip = save_shortcut.pop()
            out = self.transUp[i](out, skip, True)
            out = self.conv1x1_up[i](out)
            out = self.denseUp[i](out)

        if self.use_dappm:
            out = self.spp_low(out)
        out = self.finalConv(out)
        
        out = F.interpolate(out,
                            size=(size_in[2], size_in[3]),
                            mode="bilinear",
                            align_corners=True)
        return out

In [ ]:
# dapp in low U part and upsample to original

class hardnet_m_upsamle_high(nn.Module):
    def __init__(self, n_classes=19, use_dappm=False):
        super(hardnet_m_upsamle_high, self).__init__()
        self.n_classes = n_classes
        self.use_dappm = use_dappm

        spp_channels = 128
        first_ch  = [16,24,32,48]
        ch_list = [  64, 96, 160, 224, 320]
        grmul = 1.7
        gr       = [  10,16,18,24,32]
        n_layers = [   4, 4, 8, 8, 8]

        base = [
            ConvLayer(          3, first_ch[0], stride=2, dilation=1),
            ConvLayer(first_ch[0], first_ch[1], stride=1, dilation=2),
            ConvLayer(first_ch[1], first_ch[2], stride=2, dilation=1),
            ConvLayer(first_ch[2], first_ch[3], stride=1, dilation=2)
        ]

        self.base = nn.ModuleList(base)

        blks_count = len(n_layers)
        ch = first_ch[-1]
        skip_connection_channel_counts = []
        self.shortcut_layers = []
        for i in range(blks_count):
            block = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            ch = block.get_out_ch()
            skip_connection_channel_counts.append(ch)
            
            self.base.append(block)
            if i < blks_count-1:
                self.shortcut_layers.append(len(self.base)-1)
            

            self.base.append(ConvLayer(ch, ch_list[i], kernel=1))
            ch = ch_list[i]
            if (i < blks_count - 1):
                self.base.append(nn.AvgPool2d(kernel_size=2, stride=2))
            elif self.use_dappm:
                self.spp = DAPPM(ch_list[-1],  spp_channels, ch_list[-1])
                self.upsample_spp = TransitionUp(ch_list[-1], ch_list[-1])

        
        self.conv1x1_up = nn.ModuleList()
        self.transUp = nn.ModuleList()
        self.denseUp = nn.ModuleList()

        blks_count -= 1
        self.blks_count = blks_count 

        for i in range(blks_count - 1, -1, -1):
            self.transUp.append(TransitionUp(ch, ch))
            ch = ch + skip_connection_channel_counts[i]
            self.conv1x1_up.append(ConvLayer(ch, ch//2, kernel=1))
            ch = ch//2
            blk = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            self.denseUp.append(blk)
            ch = blk.get_out_ch()
        
        if self.use_dappm:
            # mid of network, low part of U
            ch = ch + ch_list[-1]

        self.finalConv = nn.Conv2d(ch, n_classes, kernel_size=1, 
                                   stride=1, padding=0, bias=True)

    def forward(self, x):
        size_in = x.size()
        out = x
        save_shortcut = []
        for i in range(len(self.base)):
            out = self.base[i](out)
            if i in self.shortcut_layers:
                save_shortcut.append(out)
        
        save_for_spp = out
        
        for i in range(self.blks_count):
            skip = save_shortcut.pop()
            out = self.transUp[i](out, skip, True)
            out = self.conv1x1_up[i](out)
            out = self.denseUp[i](out)

        if self.use_dappm:
            spp_res = self.spp(save_for_spp)
            out = self.upsample_spp(spp_res, out, True)
        out = self.finalConv(out)
        
        out = F.interpolate(out,
                            size=(size_in[2], size_in[3]),
                            mode="bilinear",
                            align_corners=True)
        return out

## Compare original & structure with dappm

In [ ]:
model = hardnet_m()

In [ ]:
model_dapp = hardnet_m(use_dappm=True)

In [ ]:
model_dapp_other = hardnet_m_upsamle_high(use_dappm=True)

In [ ]:
model_dapp_other

In [ ]:
summary(model_dapp_other, (3,1024, 1024))

In [ ]:
out = model(torch.rand(2,3, 1024, 1024))
out.shape

In [ ]:
out = model_dapp(torch.rand(2,3, 1024, 1024))
out.shape

In [ ]:
out = model_dapp_other(torch.rand(2,3, 1024, 1024))
out.shape

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


torch.Size([2, 19, 1024, 1024])

## Preparation

In [ ]:
!git clone https://github.com/PingoLH/FCHarDNet.git
!mv FCHarDNet/ptsemseg ./

fatal: destination path 'FCHarDNet' already exists and is not an empty directory.
mv: cannot stat 'FCHarDNet/ptsemseg': No such file or directory


In [ ]:
import yaml
from torch.utils import data

from ptsemseg.models import get_model
from ptsemseg.loss import get_loss_function
from ptsemseg.loader import get_loader
from ptsemseg.utils import get_logger
from ptsemseg.metrics import runningScore, averageMeter
from ptsemseg.augmentations import get_composed_augmentations
from ptsemseg.schedulers import get_scheduler
from ptsemseg.optimizers import get_optimizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('FCHarDNet/configs/hardnet.yml') as cf:
    cfg = yaml.load(cf)
    cfg['data'].update({
        'path':'/content/drive/MyDrive/RnD/datasets/',
        'sbd_path': '/content/drive/MyDrive/RnD/datasets/'
    })
    cfg['training'].update({
        'train_iters': 3000,
        'batch_size': 4,
        'n_workers': 2
    })
    cfg['training']['lr_schedule'].update({
        'max_iter': 240000
    })

In [ ]:
# Setup seeds
torch.manual_seed(cfg.get("seed", 1337))
torch.cuda.manual_seed(cfg.get("seed", 1337))
np.random.seed(cfg.get("seed", 1337))
random.seed(cfg.get("seed", 1337))

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup Augmentations
augmentations = cfg["training"].get("augmentations", None)
data_aug = get_composed_augmentations(augmentations)

# Setup Dataloader
data_loader = get_loader(cfg["data"]["dataset"])
data_path = cfg["data"]["path"]

t_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg["data"]["train_split"],
    img_size=(cfg["data"]["img_rows"], cfg["data"]["img_cols"]),
    augmentations=data_aug,
)
n_classes = t_loader.n_classes
trainloader = data.DataLoader(
    t_loader,
    batch_size=cfg["training"]["batch_size"],
    num_workers=cfg["training"]["n_workers"],
    shuffle=True,
)

v_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg["data"]["val_split"],
    img_size=(1024,2048),
)
valloader = data.DataLoader(
    v_loader, batch_size=cfg["training"]["batch_size"], num_workers=cfg["training"]["n_workers"]
)

Found 2975 train images
Found 500 val images


In [ ]:
running_metrics_val = runningScore(n_classes)
model = hardnet_m(use_dappm=True).to(device)
total_params = sum(p.numel() for p in model.parameters())
print( 'Parameters:',total_params )
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.apply(weights_init)
pass

Parameters: 5690117


In [ ]:
optimizer_cls = get_optimizer(cfg)
optimizer_params = {k: v for k, v in cfg["training"]["optimizer"].items() if k != "name"}
optimizer = optimizer_cls(model.parameters(), **optimizer_params)
scheduler = get_scheduler(optimizer, cfg["training"]["lr_schedule"])
loss_fn = get_loss_function(cfg)

print("Using optimizer {}".format(optimizer))
print("Using loss {}".format(loss_fn))

val_loss_meter = averageMeter()
time_meter = averageMeter()

Using optimizer SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.02
    lr: 0.02
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
Using loss functools.partial(<function bootstrapped_cross_entropy2d at 0x7f40d5304950>, min_K=4096, loss_th=0.3, size_average=True)


In [ ]:
start_iter = 0
best_iou = -100.0
i = start_iter
flag = True
loss_all = 0
loss_n = 0

In [ ]:
base_path = "/content/drive/MyDrive/RnD/runs/HarDNet/"
model_arch = 'hardnet_dappm'
model_name = f'{model_arch}_{cfg["data"]["dataset"]}'
model_modification_path = os.path.join(base_path, model_arch)

In [ ]:
logdir = os.path.join(model_modification_path, str(datetime.fromtimestamp(int(time.time()))))
writer = SummaryWriter(log_dir=logdir)

print("RUNDIR: {}".format(logdir))
# shutil.copy(args.config, logdir)

logger = get_logger(logdir)
logger.info("Let the games begin")

INFO:ptsemseg:Let the games begin


RUNDIR: /content/drive/MyDrive/RnD/runs/HarDNet/hardnet_dappm/2021-05-09 12:23:48


## Model pretrained

In [ ]:
!ls {model_modification_path}

'2021-05-09 12:20:21'  '2021-05-09 12:23:48'


In [ ]:
# runs = sorted(os.listdir(model_modification_path), reverse=True)
# best_path = None
# last_path = None
# for run in runs:
#     tmp_base = os.path.join(model_modification_path, run)
#     checkpoint = os.path.join(tmp_base, f'{model_name}_checkpoint.pkl')
#     best = os.path.join(tmp_base, f'{model_name}_best_model.pkl')
#     if not last_path and os.path.exists(checkpoint):
#         last_path = checkpoint
#     if not best_path and os.path.exists(best):
#         best_path = best
#     if last_path and best_path:
#         break

In [ ]:
# if best_path and last_path:
#     loaded = torch.load(last_path)
#     best_iou_arrc = torch.load(best_path)
#     model_state = loaded.get('model_state')

#     optimizer_state = loaded.get('optimizer_state')
#     scheduler_state = loaded.get('scheduler_state')
#     start_iter = loaded.get('epoch')

#     best_iou = best_iou_arrc.get('best_iou')
#     i = start_iter
#     flag = True
#     loss_all = 0
#     loss_n = 0

#     model.load_state_dict(model_state)
#     optimizer.load_state_dict(optimizer_state)
#     scheduler.load_state_dict(scheduler_state)

In [ ]:
# i += 1
# cfg["training"].update({
#     'train_iters': i+3000
# })
# flag = True

In [ ]:
i, cfg["training"]["train_iters"], best_iou

(0, 3000, -100.0)

## Training

In [ ]:
while i <= cfg["training"]["train_iters"] and flag:
    for (images, labels, _) in trainloader:
        i += 1
        start_ts = time.time()
        model.train()
        images = images.to(device)
        labels = labels.to(device)
 
        optimizer.zero_grad()
        outputs = model(images)
 
        loss = loss_fn(input=outputs, target=labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        c_lr = scheduler.get_lr()
 
        time_meter.update(time.time() - start_ts)
        loss_all += loss.item()
        loss_n += 1
        
        if (i + 1) % cfg["training"]["print_interval"] == 0:
            fmt_str = "Iter [{:d}/{:d}]  Loss: {:.4f}  Time/Image: {:.4f}  lr={:.6f}"
            print_str = fmt_str.format(
                i + 1,
                cfg["training"]["train_iters"],
                loss_all / loss_n,
                time_meter.avg / cfg["training"]["batch_size"],
                c_lr[0],
            )
            
 
            print(print_str)
            logger.info(print_str)
            writer.add_scalar("loss/train_loss", loss.item(), i + 1)
            time_meter.reset()
 
        if (i + 1) % cfg["training"]["val_interval"] == 0 or (i + 1) == cfg["training"][
            "train_iters"
        ]:
            torch.cuda.empty_cache()
            model.eval()
            loss_all = 0
            loss_n = 0
            with torch.no_grad():
                for i_val, (images_val, labels_val, _) in tqdm(enumerate(valloader)):
                    images_val = images_val.to(device)
                    labels_val = labels_val.to(device)
 
                    outputs = model(images_val)
                    val_loss = loss_fn(input=outputs, target=labels_val)
 
                    pred = outputs.data.max(1)[1].cpu().numpy()
                    gt = labels_val.data.cpu().numpy()
 
                    running_metrics_val.update(gt, pred)
                    val_loss_meter.update(val_loss.item())
 
            writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
            logger.info("Iter %d Val Loss: %.4f" % (i + 1, val_loss_meter.avg))
 
            score, class_iou = running_metrics_val.get_scores()
            for k, v in score.items():
                print(k, v)
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/{}".format(k), v, i + 1)
 
            for k, v in class_iou.items():
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/cls_{}".format(k), v, i + 1)
 
            val_loss_meter.reset()
            running_metrics_val.reset()
            
            state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "scheduler_state": scheduler.state_dict(),
            }
            save_path = os.path.join(
                writer.file_writer.get_logdir(),
                "{}_{}_checkpoint.pkl".format(model_arch, cfg["data"]["dataset"]),
            )
            torch.save(state, save_path)
 
            if score["Mean IoU : \t"] >= best_iou:
                best_iou = score["Mean IoU : \t"]
                state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "best_iou": best_iou,
                }
                save_path = os.path.join(
                    writer.file_writer.get_logdir(),
                    "{}_{}_best_model.pkl".format(model_arch, cfg["data"]["dataset"]),
                )
                torch.save(state, save_path)
            torch.cuda.empty_cache()
 
        if (i + 1) == cfg["training"]["train_iters"]:
            flag = False
            break

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
INFO:ptsemseg:Iter [10/3000]  Loss: 2.5497  Time/Image: 0.2424  lr=0.019999


Iter [10/3000]  Loss: 2.5497  Time/Image: 0.2424  lr=0.019999


INFO:ptsemseg:Iter [20/3000]  Loss: 2.3279  Time/Image: 0.2440  lr=0.019999


Iter [20/3000]  Loss: 2.3279  Time/Image: 0.2440  lr=0.019999


INFO:ptsemseg:Iter [30/3000]  Loss: 2.1953  Time/Image: 0.2437  lr=0.019998


...

Iter [2990/3000]  Loss: 1.3004  Time/Image: 0.2489  lr=0.019776


INFO:ptsemseg:Iter [3000/3000]  Loss: 1.2977  Time/Image: 0.2492  lr=0.019775


Iter [3000/3000]  Loss: 1.2977  Time/Image: 0.2492  lr=0.019775


125it [03:04,  1.48s/it]
INFO:ptsemseg:Iter 3000 Val Loss: 1.4394
INFO:ptsemseg:Overall Acc: 	: 0.770278949086707
INFO:ptsemseg:Mean Acc : 	: 0.2400036121198974
INFO:ptsemseg:FreqW Acc : 	: 0.6376844833120168
INFO:ptsemseg:Mean IoU : 	: 0.18321929516563504
INFO:ptsemseg:0: 0.8733742009475921
INFO:ptsemseg:1: 0.2986313244978403
INFO:ptsemseg:2: 0.5947619592647345
INFO:ptsemseg:3: 0.0
INFO:ptsemseg:4: 3.088194732812175e-05
INFO:ptsemseg:5: 0.036851660779130525
INFO:ptsemseg:6: 0.0
INFO:ptsemseg:7: 0.06672863383973798
INFO:ptsemseg:8: 0.710618581101658
INFO:ptsemseg:9: 0.21261127654198125
INFO:ptsemseg:10: 0.14649311601272066
INFO:ptsemseg:11: 0.06817809875978605
INFO:ptsemseg:12: 0.0
INFO:ptsemseg:13: 0.4706738103370599
INFO:ptsemseg:14: 0.0
INFO:ptsemseg:15: 0.0
INFO:ptsemseg:16: 0.0
INFO:ptsemseg:17: 0.0
INFO:ptsemseg:18: 0.002213064117495923


Overall Acc: 	 0.770278949086707
Mean Acc : 	 0.2400036121198974
FreqW Acc : 	 0.6376844833120168
Mean IoU : 	 0.18321929516563504
